## Mini-Project 2

In [ ]:
%run svm_author_id.py

In [ ]:
%run svm_author_id.py

In [1]:
# %load ../tools/email_preprocess
#!/usr/bin/python

import pickle
import cPickle
import numpy

from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif



def preprocess(words_file = "../tools/word_data.pkl", authors_file="../tools/email_authors.pkl"):
    """ 
        this function takes a pre-made list of email texts (by default word_data.pkl)
        and the corresponding authors (by default email_authors.pkl) and performs
        a number of preprocessing steps:
            -- splits into training/testing sets (10% testing)
            -- vectorizes into tfidf matrix
            -- selects/keeps most helpful features

        after this, the feaures and labels are put into numpy arrays, which play nice with sklearn functions

        4 objects are returned:
            -- training/testing features
            -- training/testing labels

    """

    ### the words (features) and authors (labels), already largely preprocessed
    ### this preprocessing will be repeated in the text learning mini-project
    authors_file_handler = open(authors_file, "r")
    authors = pickle.load(authors_file_handler)
    authors_file_handler.close()

    words_file_handler = open(words_file, "r")
    word_data = cPickle.load(words_file_handler)
    words_file_handler.close()

    ### test_size is the percentage of events assigned to the test set (remainder go into training)
    features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)



    ### text vectorization--go from strings to lists of numbers
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
    features_train_transformed = vectorizer.fit_transform(features_train)
    features_test_transformed  = vectorizer.transform(features_test)



    ### feature selection, because text is super high dimensional and 
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=10)
    selector.fit(features_train_transformed, labels_train)
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed  = selector.transform(features_test_transformed).toarray()

    ### info on the data
    print "no. of Chris training emails:", sum(labels_train)
    print "no. of Sara training emails:", len(labels_train)-sum(labels_train)
    
    return features_train_transformed, features_test_transformed, labels_train, labels_test


In [2]:
# %load svm_author_id.py
#!/usr/bin/python

""" 
    this is the code to accompany the Lesson 2 (SVM) mini-project

    use an SVM to identify emails from the Enron corpus by their authors
    
    Sara has label 0
    Chris has label 1

"""
    
import sys
from time import time
#sys.path.append("../tools/")
#from email_preprocess import preprocess


### features_train and features_test are the features for the training
### and testing datasets, respectively
### labels_train and labels_test are the corresponding item labels
features_train, features_test, labels_train, labels_test = preprocess()

#features_train = features_train[:len(features_train)/100] 
#labels_train = labels_train[:len(labels_train)/100] 
print len(labels_train)
print len(labels_test)
print labels_train[0:10]
print labels_test[0:10]

no. of Chris training emails: 7936
no. of Sara training emails: 7884
15820
1758
[0, 1, 0, 1, 1, 1, 0, 0, 1, 0]
[0, 0, 1, 0, 1, 0, 1, 0, 0, 0]


In [6]:
# %load svm_author_id.py
#!/usr/bin/python

""" 
    this is the code to accompany the Lesson 2 (SVM) mini-project

    use an SVM to identify emails from the Enron corpus by their authors
    
    Sara has label 0
    Chris has label 1

"""
    
import sys
from time import time
sys.path.append("../tools/")
from email_preprocess import preprocess


### features_train and features_test are the features for the training
### and testing datasets, respectively
### labels_train and labels_test are the corresponding item labels
features_train, features_test, labels_train, labels_test = preprocess()

features_train = features_train[:len(features_train)/100] 
labels_train = labels_train[:len(labels_train)/100] 
#########################################################
### your code goes here ###

from sklearn import svm

clf = svm.SVC(kernel="rbf")
to = time()
clf.fit(features_train, labels_train)
print "training time: ",round(time()-to,3),"s"
t1 = time()
pred = clf.predict(features_test)
print "testing time: ",round(time()-t1,3),"s"
from sklearn.metrics import accuracy_score
print accuracy_score(pred,labels_test)

no. of Chris training emails: 7936
no. of Sara training emails: 7884
training time:  0.103 s
testing time:  1.095 s
0.616040955631


In [3]:
# %load svm_author_id.py
#!/usr/bin/python

""" 
    this is the code to accompany the Lesson 2 (SVM) mini-project

    use an SVM to identify emails from the Enron corpus by their authors
    
    Sara has label 0
    Chris has label 1

"""
    
import sys
from time import time
sys.path.append("../tools/")
from email_preprocess import preprocess


### features_train and features_test are the features for the training
### and testing datasets, respectively
### labels_train and labels_test are the corresponding item labels
features_train, features_test, labels_train, labels_test = preprocess()

#features_train = features_train[:len(features_train)/100] 
#labels_train = labels_train[:len(labels_train)/100] 
print len(labels_train)
print len(labels_test)
#########################################################
### your code goes here ###

from sklearn import svm

C_list = [10000.0]#[10.0,100.0,1000.0,10000.0]

for C_val in C_list:
    print "C = ",C_val
    clf = svm.SVC(C=C_val,kernel="rbf")
    to = time()
    clf.fit(features_train, labels_train)
    print "training time: ",round(time()-to,3),"s"
    t1 = time()
    pred = clf.predict(features_test)
    print "testing time: ",round(time()-t1,3),"s"
    from sklearn.metrics import accuracy_score
    print accuracy_score(pred,labels_test)
#########################################################




no. of Chris training emails: 7936
no. of Sara training emails: 7884
15820
1758
C =  10000.0
training time:  104.488 s
testing time:  10.638 s
0.990898748578


In [4]:
print pred[10]," ",pred[9]
print pred[26]," ",pred[25]
print pred[50]," ",pred[49]

1   0
0   1
1   0


In [5]:
#list(pred)
print len(pred)
print "Sarah: ",list(pred).count(0)
print "Chris: ",list(pred).count(1)

1758
Sarah:  881
Chris:  877


In [ ]:
print len(pred)
print "Sarah: ",list(pred).count(0)
print "Chris: ",list(pred).count(1)